In [1]:
import pandas
import numpy

In [2]:
def format_cnpj(original_number):
    if original_number != "-":
        formatted_cnpj = original_number[:2] + '.' + original_number[2:5] + '.' 
        return(formatted_cnpj + original_number[5:8] + '/' + original_number[8:12] + '-' + original_number[12:])
    else:
        return(original_number)

data_economatica = pandas.read_csv("acoes_economatica.csv", encoding = "utf-8", sep = ";", dtype=str)
data_economatica["CNPJ"] = data_economatica["CNPJ"].str.replace(",","", regex = False)
data_economatica["CNPJ"][data_economatica["CNPJ"] != "-"] = data_economatica["CNPJ"].str.zfill(14)
data_economatica["CNPJ"] = data_economatica["CNPJ"].astype(str).apply(format_cnpj)
data_economatica.drop(["Nome.1"], axis = 1, inplace = True)
data_economatica.columns = ["NAME", "CLASS", "CODE", "CNPJ", "STOCK_TYPE", "ISIN", "SETOR(BOVESPA)", "SUBSETOR(BOVESPA)", "SEGMENTO(BOVESPA)"]
data_economatica = data_economatica[~(data_economatica.CODE.str.strip().str.contains(" "))]
data_economatica.CNPJ[(data_economatica.CNPJ == "-")] = data_economatica.CODE
data_economatica = data_economatica.map(lambda x: x.upper() if isinstance(x, str) else x)
data_economatica

,NAME,CLASS,CODE,CNPJ,STOCK_TYPE,ISIN,SETOR(BOVESPA),SUBSETOR(BOVESPA),SEGMENTO(BOVESPA)
2,3R PETROLEUM,ON,RRRP3,12.091.809/0001-55,AÇÃO,BRRRRPACNOR5,PETRÓLEO GÁS E BIOCOMBUSTÍVEIS,PETRÓLEO GÁS E BIOCOMBUSTÍVEIS,EXPLORAÇÃO REFINO E DISTRIBUIÇÃO
3,3TENTOS,ON,TTEN3,94.813.102/0001-70,AÇÃO,BRTTENACNOR0,CONSUMO NÃO CÍCLICO,AGROPECUÁRIA,AGRICULTURA
4,521 PARTICIP,ON,QVUM3B,01.547.749/0001-16,AÇÃO,-,-,-,-
5,524 PARTICIP,ON,QVQP3,01.851.771/0001-55,AÇÃO,BRQVQPACNOR1,OUTROS,OUTROS,OUTROS
6,A P PARTICIPACOES,ON,APPA3,02.288.752/0001-25,AÇÃO,-,-,-,-
...,...,...,...,...,...,...,...,...,...
1949,YDUQS PART,PN,ESTC4,08.807.432/0001-10,AÇÃO,-,CONSUMO CÍCLICO,DIVERSOS,SERVIÇOS EDUCACIONAIS
1950,YDUQS PART,UNT N2,ESTC11,08.807.432/0001-10,AÇÃO,-,CONSUMO CÍCLICO,DIVERSOS,SERVIÇOS EDUCACIONAIS
1954,ZAIN PART,ON,OPZI3B,02.363.918/0001-20,AÇÃO,BROPZIACNOR4,FINANCEIRO,OUTROS,OUTROS
1955,ZAMP S.A.,ON,ZAMP3,13.574.594/0001-96,AÇÃO,BRZAMPACNOR5,CONSUMO CÍCLICO,HOTÉIS E RESTAURANTES,RESTAURANTE E SIMILARES


In [3]:
data_economatica[data_economatica.CODE.str.startswith("PETR")]

,NAME,CLASS,CODE,CNPJ,STOCK_TYPE,ISIN,SETOR(BOVESPA),SUBSETOR(BOVESPA),SEGMENTO(BOVESPA)
1351,PETROBRAS,ON,PETR3,33.000.167/0001-01,AÇÃO,BRPETRACNOR9,PETRÓLEO GÁS E BIOCOMBUSTÍVEIS,PETRÓLEO GÁS E BIOCOMBUSTÍVEIS,EXPLORAÇÃO REFINO E DISTRIBUIÇÃO
1352,PETROBRAS,PN,PETR4,33.000.167/0001-01,AÇÃO,BRPETRACNPR6,PETRÓLEO GÁS E BIOCOMBUSTÍVEIS,PETRÓLEO GÁS E BIOCOMBUSTÍVEIS,EXPLORAÇÃO REFINO E DISTRIBUIÇÃO


In [4]:
def for_col(df):
    x = pandas.DataFrame("-", index = ["x"], columns = df.columns)
    for col in df.columns:
        y = df[col].unique().tolist()
        if len(y) == 1: 
            x.loc["x",col] = y[0]
        else:
            x.loc["x",col] = ';'.join(y)
    return(x)

dataset = pandas.DataFrame("-", index = data_economatica.CNPJ.unique(), columns = data_economatica.columns)
for cnpj in dataset.index:
    aux_df = data_economatica[data_economatica.CNPJ == cnpj]
    if aux_df.shape[0] == 1:
        dataset.loc[cnpj,:] = aux_df.iloc[0,:].tolist()
    else:
        dataset.loc[cnpj,:] = for_col(df = aux_df).iloc[0,:].tolist()
dataset

,NAME,CLASS,CODE,CNPJ,STOCK_TYPE,ISIN,SETOR(BOVESPA),SUBSETOR(BOVESPA),SEGMENTO(BOVESPA)
12.091.809/0001-55,3R PETROLEUM,ON,RRRP3,12.091.809/0001-55,AÇÃO,BRRRRPACNOR5,PETRÓLEO GÁS E BIOCOMBUSTÍVEIS,PETRÓLEO GÁS E BIOCOMBUSTÍVEIS,EXPLORAÇÃO REFINO E DISTRIBUIÇÃO
94.813.102/0001-70,3TENTOS,ON,TTEN3,94.813.102/0001-70,AÇÃO,BRTTENACNOR0,CONSUMO NÃO CÍCLICO,AGROPECUÁRIA,AGRICULTURA
01.547.749/0001-16,521 PARTICIP,ON,QVUM3B,01.547.749/0001-16,AÇÃO,-,-,-,-
01.851.771/0001-55,524 PARTICIP,ON,QVQP3,01.851.771/0001-55,AÇÃO,BRQVQPACNOR1,OUTROS,OUTROS,OUTROS
02.288.752/0001-25,A P PARTICIPACOES,ON;PN,APPA3;APPA4,02.288.752/0001-25,AÇÃO,-,-,-,-
...,...,...,...,...,...,...,...,...,...
02.217.319/0001-07,YBYRA S/A,ON,YBRA3,02.217.319/0001-07,AÇÃO,BRYBRAACNOR7,OUTROS,OUTROS,OUTROS
08.807.432/0001-10,YDUQS PART,ON;PN;UNT N2,YDUQ3;ESTC4;ESTC11,08.807.432/0001-10,AÇÃO,BRYDUQACNOR3;-,CONSUMO CÍCLICO,DIVERSOS,SERVIÇOS EDUCACIONAIS
02.363.918/0001-20,ZAIN PART,ON,OPZI3B,02.363.918/0001-20,AÇÃO,BROPZIACNOR4,FINANCEIRO,OUTROS,OUTROS
13.574.594/0001-96,ZAMP S.A.,ON,ZAMP3,13.574.594/0001-96,AÇÃO,BRZAMPACNOR5,CONSUMO CÍCLICO,HOTÉIS E RESTAURANTES,RESTAURANTE E SIMILARES


In [5]:
dataset[dataset.CODE.str.startswith("PETR")]

,NAME,CLASS,CODE,CNPJ,STOCK_TYPE,ISIN,SETOR(BOVESPA),SUBSETOR(BOVESPA),SEGMENTO(BOVESPA)
33.000.167/0001-01,PETROBRAS,ON;PN,PETR3;PETR4,33.000.167/0001-01,AÇÃO,BRPETRACNOR9;BRPETRACNPR6,PETRÓLEO GÁS E BIOCOMBUSTÍVEIS,PETRÓLEO GÁS E BIOCOMBUSTÍVEIS,EXPLORAÇÃO REFINO E DISTRIBUIÇÃO


In [6]:
empresas_cvm = pandas.read_csv("EmpresasBrasilCVM.txt")
empresas_cvm["CVM_CODE"] = empresas_cvm["CVM_CODE"].astype(str).str.zfill(6)
empresas_cvm["SOURCE_CVM"] = True
empresas_cvm

,CNPJ,FIRM_NAME,CVM_FIRM_TYPE,CVM_CODE,CVM_STATUS,SOURCE_CVM
0,31.694.680/0001-14,A C M AGRICOLA SA,CIAS INCENTIVADAS REGISTRADAS,505480,Cancelado em 19/11/2008,True
1,08.690.448/0001-96,A G HOTEIS E TURISMO SA,CIAS INCENTIVADAS REGISTRADAS,502316,Cancelado em 24/08/2007,True
2,92.659.614/0001-06,A J RENNER SA IND E PARTIP,CIAS ABERTAS,000035,Cancelado em 17/06/1998,True
3,04.895.165/0001-20,A M FIDALGO SA MAT DE CONSTR,CIAS INCENTIVADAS REGISTRADAS,504076,Cancelado em 28/09/2008,True
4,06.932.909/0001-64,A O GASPAR INDS SA,CIAS INCENTIVADAS REGISTRADAS,503592,Cancelado em 14/02/2012,True
...,...,...,...,...,...,...
4636,11.396.633/0001-87,3A COMPANHIA SECURITIZADORA,CIAS ABERTAS,021954,Cancelado em 18/12/2015,True
4637,12.091.809/0001-55,3R PETROLEUM OLÉO E GÁS S.A.,CIAS ABERTAS,025291,Concedido em 09/11/2020,True
4638,01.547.749/0001-16,521 PARTICIPAÇOES S.A. - EM LIQUIDAÇÃO EXTRAJU...,CIAS ABERTAS,016330,Concedido em 11/07/1997,True
4639,01.851.771/0001-55,524 PARTICIPAÇOES SA,CIAS ABERTAS,016284,Concedido em 30/05/1997,True


In [7]:
brfin = pandas.read_csv("CodigosDocsCVM.txt")
brfin["CVMCode"] = brfin["CVMCode"].astype(str).str.zfill(6)
#brfin = brfin[~brfin.CVMCode.isin(empresas_cvm.CVM_CODE)]
brfin.drop(["CVMStatus"], axis = 1, inplace = True)
brfin.columns = ["CVM_CODE", "FIRM_NAME"]
brfin["SOURCE_BRFIN"] = True
brfin

,CVM_CODE,FIRM_NAME,SOURCE_BRFIN
0,018554,"CRÉDITO, FINANCIAMENTO E INVESTIMENTO",True
1,900049,1461 INVESTIMENTOS S.A,True
2,252224,2W ECOBANK S.A.,True
3,025224,2W ECOBANK S.A.,True
4,917581,2W ENERGIA S.A.,True
...,...,...,...
5780,056456,ZOOM VIDEO COMMUNICATIONS INC,True
5781,058955,ZOOMINFO TECHNOLOGIES INC,True
5782,058963,ZSCALER INC,True
5783,057169,ZTO EXPRESS CAYMAN INC,True


In [8]:
df = pandas.concat([empresas_cvm, brfin[~brfin.CVM_CODE.isin(empresas_cvm.CVM_CODE)]], axis = 0)
df.SOURCE_BRFIN[(df.CVM_CODE.isin(brfin.CVM_CODE)) & (pandas.isna(df.CVM_CODE))] = True
df.SOURCE_CVM[pandas.isna(df["SOURCE_CVM"])] = False
df.SOURCE_BRFIN[pandas.isna(df["SOURCE_BRFIN"])] = False
for c in df.columns:
    df[c][pandas.isna(df[c])] = "-"
df

,CNPJ,FIRM_NAME,CVM_FIRM_TYPE,CVM_CODE,CVM_STATUS,SOURCE_CVM,SOURCE_BRFIN
0,31.694.680/0001-14,A C M AGRICOLA SA,CIAS INCENTIVADAS REGISTRADAS,505480,Cancelado em 19/11/2008,True,False
1,08.690.448/0001-96,A G HOTEIS E TURISMO SA,CIAS INCENTIVADAS REGISTRADAS,502316,Cancelado em 24/08/2007,True,False
2,92.659.614/0001-06,A J RENNER SA IND E PARTIP,CIAS ABERTAS,000035,Cancelado em 17/06/1998,True,False
3,04.895.165/0001-20,A M FIDALGO SA MAT DE CONSTR,CIAS INCENTIVADAS REGISTRADAS,504076,Cancelado em 28/09/2008,True,False
4,06.932.909/0001-64,A O GASPAR INDS SA,CIAS INCENTIVADAS REGISTRADAS,503592,Cancelado em 14/02/2012,True,False
...,...,...,...,...,...,...,...
5780,-,ZOOM VIDEO COMMUNICATIONS INC,-,056456,-,False,True
5781,-,ZOOMINFO TECHNOLOGIES INC,-,058955,-,False,True
5782,-,ZSCALER INC,-,058963,-,False,True
5783,-,ZTO EXPRESS CAYMAN INC,-,057169,-,False,True


In [9]:
data = dataset.copy()
data.index = dataset.CNPJ

data["CVM_CODE"] = "-"
data["CVM_STATUS"] = "-"
data["CVM_FIRM_TYPE"] = "-"
data["FIRM_NAME"] = "-"
data["SOURCE_ECONOMATICA"] = True
data["SOURCE_CVM"] = False
data["SOURCE_BRFIN"] = False

fetch_cols = ["CVM_CODE", "CVM_STATUS", "CVM_FIRM_TYPE", "FIRM_NAME", "SOURCE_CVM", "SOURCE_BRFIN"]

df_in = df[(df.CNPJ.isin(data.index)) & (df.CNPJ != "-")]
df_in.index = df_in.CNPJ

data.loc[df_in.index, fetch_cols] = df_in[fetch_cols].values

data

,NAME,CLASS,CODE,CNPJ,STOCK_TYPE,ISIN,SETOR(BOVESPA),SUBSETOR(BOVESPA),SEGMENTO(BOVESPA),CVM_CODE,CVM_STATUS,CVM_FIRM_TYPE,FIRM_NAME,SOURCE_ECONOMATICA,SOURCE_CVM,SOURCE_BRFIN
CNPJ,,,,,,,,,,,,,,,,
12.091.809/0001-55,3R PETROLEUM,ON,RRRP3,12.091.809/0001-55,AÇÃO,BRRRRPACNOR5,PETRÓLEO GÁS E BIOCOMBUSTÍVEIS,PETRÓLEO GÁS E BIOCOMBUSTÍVEIS,EXPLORAÇÃO REFINO E DISTRIBUIÇÃO,025291,Concedido em 09/11/2020,CIAS ABERTAS,3R PETROLEUM OLÉO E GÁS S.A.,True,True,False
94.813.102/0001-70,3TENTOS,ON,TTEN3,94.813.102/0001-70,AÇÃO,BRTTENACNOR0,CONSUMO NÃO CÍCLICO,AGROPECUÁRIA,AGRICULTURA,025950,Concedido em 21/06/2021,CIAS ABERTAS,TRÊS TENTOS AGROINDUSTRIAL S.A.,True,True,False
01.547.749/0001-16,521 PARTICIP,ON,QVUM3B,01.547.749/0001-16,AÇÃO,-,-,-,-,016330,Concedido em 11/07/1997,CIAS ABERTAS,521 PARTICIPAÇOES S.A. - EM LIQUIDAÇÃO EXTRAJU...,True,True,False
01.851.771/0001-55,524 PARTICIP,ON,QVQP3,01.851.771/0001-55,AÇÃO,BRQVQPACNOR1,OUTROS,OUTROS,OUTROS,016284,Concedido em 30/05/1997,CIAS ABERTAS,524 PARTICIPAÇOES SA,True,True,False
02.288.752/0001-25,A P PARTICIPACOES,ON;PN,APPA3;APPA4,02.288.752/0001-25,AÇÃO,-,-,-,-,016802,Cancelado em 23/12/2004,CIAS ABERTAS,A.P. PARTICIPAÇOES SA,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
02.217.319/0001-07,YBYRA S/A,ON,YBRA3,02.217.319/0001-07,AÇÃO,BRYBRAACNOR7,OUTROS,OUTROS,OUTROS,016705,Concedido em 08/12/1997,CIAS ABERTAS,YBYRA CAPITAL S.A,True,True,False
08.807.432/0001-10,YDUQS PART,ON;PN;UNT N2,YDUQ3;ESTC4;ESTC11,08.807.432/0001-10,AÇÃO,BRYDUQACNOR3;-,CONSUMO CÍCLICO,DIVERSOS,SERVIÇOS EDUCACIONAIS,021016,Concedido em 26/07/2007,CIAS ABERTAS,YDUQS PARTICIPACOES S.A.,True,True,False
02.363.918/0001-20,ZAIN PART,ON,OPZI3B,02.363.918/0001-20,AÇÃO,BROPZIACNOR4,FINANCEIRO,OUTROS,OUTROS,017361,Cancelado em 14/08/2020,CIAS ABERTAS,ZAIN PARTICIPAÇÕES S/A,True,True,False


In [10]:
data[data.CODE.str.startswith("PETR")]

,NAME,CLASS,CODE,CNPJ,STOCK_TYPE,ISIN,SETOR(BOVESPA),SUBSETOR(BOVESPA),SEGMENTO(BOVESPA),CVM_CODE,CVM_STATUS,CVM_FIRM_TYPE,FIRM_NAME,SOURCE_ECONOMATICA,SOURCE_CVM,SOURCE_BRFIN
CNPJ,,,,,,,,,,,,,,,,
33.000.167/0001-01,PETROBRAS,ON;PN,PETR3;PETR4,33.000.167/0001-01,AÇÃO,BRPETRACNOR9;BRPETRACNPR6,PETRÓLEO GÁS E BIOCOMBUSTÍVEIS,PETRÓLEO GÁS E BIOCOMBUSTÍVEIS,EXPLORAÇÃO REFINO E DISTRIBUIÇÃO,009512,Concedido em 20/07/1977,CIAS ABERTAS,PETRÓLEO BRASILEIRO S.A. - PETROBRAS,True,True,False


In [11]:
data.to_csv("Brazil.txt", index = False)